In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2024-09-06 12:24:04.219595: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-06 12:24:04.316925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 12:24:05.858225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.13.1


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28),name='bitmap'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2669 - accuracy: 0.9226
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1257 - accuracy: 0.9624
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0910 - accuracy: 0.9724
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0721 - accuracy: 0.9782
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0586 - accuracy: 0.9821


In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0757 - accuracy: 0.9762 - 605ms/epoch - 2ms/step


[0.0757015123963356, 0.9761999845504761]

In [29]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax(name="probabilities")
])

In [30]:
probability_model.inputs[0].shape

TensorShape([None, 28, 28])

In [31]:
probability_model.outputs[0].shape

TensorShape([None, 10])

In [32]:
probability_model(x_test[4:5])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[2.7840372e-07, 1.2425036e-08, 2.9559276e-06, 3.2774145e-07,
        9.9928302e-01, 1.3890549e-06, 1.3037358e-06, 1.3159700e-04,
        2.0548114e-06, 5.7697902e-04]], dtype=float32)>

In [33]:
!pip install onnx tf2onnx


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [34]:
import tensorflow as tf
import tf2onnx
import onnx


In [38]:
model.output_names=['output']
input_signature = [tf.TensorSpec(probability_model.inputs[0].shape, probability_model.inputs[0].dtype, name='bitmap')]

onnx_model, _ = tf2onnx.convert.from_keras(probability_model, input_signature, opset=13)
onnx.save(onnx_model, "model.onnx")

Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-09-06 12:18:17.152656: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-09-06 12:18:17.152912: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-09-06 12:18:17.184386: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-09-06 12:18:17.184572: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
